## Imports:

In [1]:
import scipy.io
import os
import pandas
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pylab as plt_lab
import matplotlib.gridspec as gridspec
import obspy
import pandas as pd

RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd

In [2]:
import warnings
warnings.filterwarnings('ignore')

## Load data:
We are going to use this data to plot spectograms of SCG signals. 

In [3]:
path = "/Users/ecem/Desktop/phonocardiogram/data/MV"

In [4]:
waves = np.load(path + "/MV_waves-10sec.npy", allow_pickle= True)


In [5]:
print(waves.shape)


(10719, 40000)


## Zero Crossing Rate:
Compute the zero-crossing rate of an audio time series.
The zero-crossing rate (ZCR) is the rate at which a signal changes from positive to zero to negative or from negative to zero to positive.

In [6]:
import librosa as lbs

In [7]:
lbs.feature.zero_crossing_rate(waves[7],frame_length= 8000, hop_length = 12000)

array([[0.01825 , 0.035375, 0.034   , 0.02975 ]])

In [8]:
lbs.feature.zero_crossing_rate(waves[7], frame_length= 8000, hop_length = 12000).shape

(1, 4)

In [9]:
def zero_crossing(array, frame_length= 2048, hl = 512):
    zero_crossing_rate = np.ones(shape = (array.shape[0], 5)) #buradaki ikinci sayı hop lenght e göre değişmeli 

    for i in range(array.shape[0]):
        zero_crossing_rate[i][0:4] = lbs.feature.zero_crossing_rate(array[i],  frame_length = frame_length,
                                                                    hop_length = hl)
        zero_crossing_rate[i][4] = sum(lbs.zero_crossings(array[i]))
    return zero_crossing_rate

        

In [10]:
zcrs = zero_crossing(waves, 8000, hl = 12000) #çok uzun sürüyor, ya datayı kısaltmalıyız ya da hop lenght artmalı

In [11]:
columns_zcr = []
for i in range(zcrs.shape[1]-1):
    columns_zcr.append("zcrs{}".format(i))

    
columns_zcr.append("#ofcrossing")

print(columns_zcr)

['zcrs0', 'zcrs1', 'zcrs2', 'zcrs3', '#ofcrossing']


In [12]:
zcrs[7]

array([1.8250e-02, 3.5375e-02, 3.4000e-02, 2.9750e-02, 1.3510e+03])

In [13]:
def dataframe_creator(array, df = pd.DataFrame(), l = 7, columns = None):
    values = []

    for i in range(array.shape[0]):
        llist = list(array[i][0:l-1].tolist())
        llist.append(array[i][l-1])
        
        values.append(llist)
        medium = pd.DataFrame([values[i]], columns = columns)
        
        
        df = pd.concat([df, medium], axis = 0)
    return df 

In [14]:
df = dataframe_creator(zcrs, l =5,  columns = columns_zcr)

df.reset_index(drop = True, inplace = True)

In [15]:
df.head()

,zcrs0,zcrs1,zcrs2,zcrs3,#ofcrossing
0,0.018000,0.031625,0.035125,0.029000,1337.0
1,0.014500,0.032625,0.024375,0.025875,1148.0
2,0.016125,0.029625,0.026875,0.024750,1135.0
3,0.020625,0.035125,0.033875,0.032500,1322.0
4,0.015875,0.036875,0.038000,0.033375,1344.0


In [16]:
df.to_csv(path + '/zcrs_MV.csv')

## with Özgün Ozan Nacitarhan code

In [17]:
def zcr(array):
    zcr = np.ones(shape = (array.shape[0],1))
    for i in range(array.shape[0]):
        data_zeros = np.zeros(len(array[i]))
        data_zeros[1:] = array[i][:len(array[i]) - 1]
        zcr[i][0] = (1 / (2 * len(array[i]))) * np.sum(np.abs(np.sign(array[i]) - np.sign(data_zeros)))
        
    return zcr


In [18]:
zcrs_özgün = zcr(waves)

In [19]:
zcrs_özgün.shape

(10719, 1)

In [20]:
zcrs_özgün[0]

array([0.0334125])

In [21]:
np.save(path + "/zcrs_özgün_MV", zcrs_özgün, allow_pickle= True)

### Chromogram:
We can use Chroma feature visualization to know how dominant the characteristics of a certain pitch {C, C♯, D, D♯, E, F, F♯, G, G♯, A, A♯, B} is present in the sampled frame.

Compute a chromagram from a waveform or power spectrogram.
It returns Normalized energy for each chroma bin at each frame.

Hop size should refer to the number of samples in between successive frames. For signal analysis Hop Size should be less than the frame size, so that frames overlap.

In [22]:
import warnings
warnings.filterwarnings('ignore')

In [23]:
import librosa.display

In [24]:
lbs.feature.chroma_stft(waves[0], sr = 4000, hop_length = 12000).shape

(12, 4)

In [25]:
# we are taking time average of each pitch values.
def chromagram_average(array, sampling_rate = 256):
    pitch = np.ones(shape = (array.shape[0], 12)) 
    
    for i in range(array.shape[0]):
        for j in range(12):
            pitch[i][j] = lbs.feature.chroma_stft(np.array(array[i]), sr= sampling_rate, hop_length = 12000)[j].mean()
    return pitch

In [26]:
chro = chromagram_average(waves, sampling_rate = 4000) # bu da çok uzun sürüyor.

In [27]:
print(chro.shape)

(10719, 12)


In [28]:
lst = ["C", "C♯", "D", "D♯", "E", "F", "F♯", "G", "G♯", "A", "A♯", "B"]

In [29]:
col_chro = []
for i in range(chro.shape[1]):
    col_chro.append(lst[i])
    
print(col_chro)
print(len(col_chro))

['C', 'C♯', 'D', 'D♯', 'E', 'F', 'F♯', 'G', 'G♯', 'A', 'A♯', 'B']
12


In [30]:
ch_df = dataframe_creator(chro,
                         df =pd.DataFrame(columns = col_chro), columns = col_chro, l = 12)

ch_df.reset_index(drop = True, inplace = True)
ch_df

,C,C♯,D,D♯,E,F,F♯,G,G♯,A,A♯,B
0,0.808428,0.652943,0.501026,0.411353,0.472592,0.516226,0.536706,0.533911,0.702158,0.749642,0.66438,0.629084
1,0.693731,0.77702,0.510975,0.479124,0.398433,0.406758,0.462523,0.419838,0.448998,0.48601,0.679584,0.663704
2,0.773602,0.936913,0.699597,0.42843,0.353148,0.330228,0.269443,0.336155,0.398033,0.464294,0.556676,0.604561
3,0.878625,0.853817,0.705735,0.553189,0.554464,0.509743,0.487549,0.470062,0.471314,0.441334,0.538502,0.693372
4,0.816637,0.730825,0.546518,0.437123,0.566929,0.457746,0.434965,0.521014,0.443097,0.467492,0.598752,0.754699
...,...,...,...,...,...,...,...,...,...,...,...,...
10714,0.774699,0.781516,0.607901,0.398823,0.347227,0.421595,0.403844,0.360063,0.382821,0.525848,0.373043,0.402533
10715,0.801259,0.819637,0.8101,0.85162,0.660284,0.624944,0.616555,0.482968,0.585966,0.61571,0.679161,0.707453
10716,0.475532,0.386038,0.403829,0.480387,0.602,0.372674,0.334681,0.256551,0.309038,0.494605,0.688533,0.817985
10717,0.368217,0.548284,0.630973,0.532328,0.293125,0.427472,0.301166,0.237313,0.344682,0.398755,0.3433,0.246092


In [31]:
ch_df.to_csv(path + "/chromogram_MV.csv")

## Tempogram

Compute the tempogram: local autocorrelation of the onset strength envelope.

# kendime not tempogram değerlerini spektogram değerleri gibi kaydet.

In [32]:
def tempo_average(array): # 192,79
    means= []
    for i in range(array.shape[0]): #192
        means.append(array[i].mean())
    return np.array(means) # 192
        

In [33]:
def tempogram(array = 0, hop_lenght = 12000, sampling_rate = 256):
    
    oenv = librosa.onset.onset_strength(y=np.array(array), 
                                            sr=sampling_rate, 
                                        lag =3)

    times = librosa.times_like(oenv, sr=sampling_rate, hop_length=12000)

    tempogram = librosa.feature.tempogram(onset_envelope = oenv, sr = sampling_rate,
                                              hop_length = hop_lenght, win_length = 192 )
    # Estimate the global tempo for display purposes
    #tempo = librosa.beat.tempo(onset_envelope=oenv, sr= sampling_rate,
                                   #hop_length=hop_length)[0]
    return tempogram

In [34]:
tempogram(waves[0]).shape

(192, 79)

In [35]:
tempogram(waves[0])[0]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [36]:
tempogram(waves[0])[1]

array([0.79643789, 0.79553885, 0.79463311, 0.79372084, 0.79280224,
       0.79187752, 0.7909469 , 0.79001061, 0.78906888, 0.78812196,
       0.78717009, 0.78621354, 0.78525255, 0.78428739, 0.78331832,
       0.7823456 , 0.7813695 , 0.78039027, 0.77940818, 0.77842349,
       0.77743647, 0.77644736, 0.77545643, 0.77446395, 0.77347016,
       0.77247533, 0.77147972, 0.77048357, 0.76948716, 0.76849073,
       0.76749455, 0.76649889, 0.765504  , 0.76451014, 0.7635176 ,
       0.76252664, 0.76153754, 0.76055058, 0.75956605, 0.75858424,
       0.75760546, 0.75663001, 0.7556582 , 0.75469037, 0.75372684,
       0.75276797, 0.75181411, 0.75086563, 0.74992292, 0.74898636,
       0.74805638, 0.7471334 , 0.74621787, 0.74531025, 0.74441103,
       0.74352071, 0.74263981, 0.74176888, 0.7409085 , 0.74005926,
       0.73922179, 0.73839673, 0.73758478, 0.73678663, 0.73600303,
       0.73523476, 0.73448263, 0.73374749, 0.73303022, 0.73233173,
       0.73165301, 0.73099505, 0.73035891, 0.72974567, 0.72915

In [37]:
tempo_average(tempogram(waves[0]))

array([ 1.00000000e+00,  7.59544300e-01,  4.25784308e-01,  3.14372852e-01,
        3.43709074e-01,  3.54566055e-01,  3.21769443e-01,  2.86459061e-01,
        3.16388179e-01,  3.90483344e-01,  4.30614439e-01,  4.32041827e-01,
        4.47616509e-01,  4.70512309e-01,  4.62316598e-01,  4.17852468e-01,
        3.61060192e-01,  3.13143597e-01,  2.96191168e-01,  3.09006087e-01,
        3.11524670e-01,  2.92166783e-01,  2.69427370e-01,  2.90936136e-01,
        3.61285824e-01,  3.76069458e-01,  2.90414592e-01,  2.29926249e-01,
        2.53299212e-01,  2.96172202e-01,  2.83283195e-01,  2.21935205e-01,
        1.97717178e-01,  3.13103363e-01,  4.21944809e-01,  3.51512388e-01,
        2.19554965e-01,  1.75047827e-01,  1.97473349e-01,  1.99141358e-01,
        1.80342926e-01,  1.66048867e-01,  1.66558450e-01,  1.78381145e-01,
        1.89673451e-01,  1.87822335e-01,  1.78263509e-01,  1.78356295e-01,
        1.81126323e-01,  1.83632604e-01,  1.75064995e-01,  1.46125033e-01,
        1.22661389e-01,  

# tempogram(waves[0])[0]

In [38]:
def temp_values(array):
    final = np.ones(shape = (array.shape[0], 192))
    for i in range(array.shape[0]): #8bin civarı
        
        _x = tempogram(array[i]) #1,4000   #192,79
        #print(_x.shape)
        final[i] = tempo_average(_x) #192
        
        
    return final

In [39]:
temp = temp_values(waves) #çok uzun sürüyor 


In [40]:
temp.shape

(10719, 192)

In [41]:
columns_t = []
for i in range(temp.shape[1]):
    columns_t.append("tempogram_{}".format(i))
print(columns_t)

['tempogram_0', 'tempogram_1', 'tempogram_2', 'tempogram_3', 'tempogram_4', 'tempogram_5', 'tempogram_6', 'tempogram_7', 'tempogram_8', 'tempogram_9', 'tempogram_10', 'tempogram_11', 'tempogram_12', 'tempogram_13', 'tempogram_14', 'tempogram_15', 'tempogram_16', 'tempogram_17', 'tempogram_18', 'tempogram_19', 'tempogram_20', 'tempogram_21', 'tempogram_22', 'tempogram_23', 'tempogram_24', 'tempogram_25', 'tempogram_26', 'tempogram_27', 'tempogram_28', 'tempogram_29', 'tempogram_30', 'tempogram_31', 'tempogram_32', 'tempogram_33', 'tempogram_34', 'tempogram_35', 'tempogram_36', 'tempogram_37', 'tempogram_38', 'tempogram_39', 'tempogram_40', 'tempogram_41', 'tempogram_42', 'tempogram_43', 'tempogram_44', 'tempogram_45', 'tempogram_46', 'tempogram_47', 'tempogram_48', 'tempogram_49', 'tempogram_50', 'tempogram_51', 'tempogram_52', 'tempogram_53', 'tempogram_54', 'tempogram_55', 'tempogram_56', 'tempogram_57', 'tempogram_58', 'tempogram_59', 'tempogram_60', 'tempogram_61', 'tempogram_62', '

In [42]:
len(columns_t)

192

In [43]:
df_t = dataframe_creator(temp,
                         df =pd.DataFrame(columns = columns_t), columns = columns_t, l = 192)

df_t.reset_index(drop = True, inplace = True)
df_t

,tempogram_0,tempogram_1,tempogram_2,tempogram_3,tempogram_4,tempogram_5,tempogram_6,tempogram_7,tempogram_8,tempogram_9,...,tempogram_182,tempogram_183,tempogram_184,tempogram_185,tempogram_186,tempogram_187,tempogram_188,tempogram_189,tempogram_190,tempogram_191
0,1.0,0.759544,0.425784,0.314373,0.343709,0.354566,0.321769,0.286459,0.316388,0.390483,...,-0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0
1,1.0,0.829675,0.591039,0.498409,0.465651,0.428465,0.402594,0.398465,0.418667,0.478274,...,-0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.863447,0.630128,0.492331,0.454018,0.450235,0.457208,0.460913,0.465491,0.466925,...,0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0
3,1.0,0.872466,0.723278,0.634329,0.618567,0.646108,0.680276,0.700787,0.670776,0.632968,...,0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0
4,1.0,0.888843,0.781121,0.6982,0.670534,0.658058,0.655847,0.633783,0.599768,0.569725,...,-0.0,-0.0,0.0,0.0,0.0,-0.0,-0.0,-0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10714,1.0,0.823203,0.618276,0.553598,0.56727,0.561299,0.53557,0.543573,0.546527,0.536406,...,-0.0,0.0,0.0,-0.0,0.0,-0.0,-0.0,0.0,-0.0,0.0
10715,1.0,0.894588,0.730461,0.647374,0.624979,0.595459,0.559293,0.546308,0.569438,0.619338,...,-0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0
10716,1.0,0.932881,0.850307,0.815248,0.816232,0.808791,0.780885,0.748102,0.721469,0.715368,...,-0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0
10717,1.0,0.928896,0.845047,0.781449,0.748838,0.738295,0.705422,0.674758,0.66039,0.655627,...,0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0


In [44]:
df_t['tempogram_188'][10]

3.0387453249530185e-18

In [45]:
df_t.to_csv(path + '/tempogram_MV.csv')

In [46]:
df_t.isin([0]).sum(axis=0)

tempogram_0      0
tempogram_1      0
tempogram_2      0
tempogram_3      0
tempogram_4      0
                ..
tempogram_187    0
tempogram_188    0
tempogram_189    0
tempogram_190    0
tempogram_191    0
Length: 192, dtype: int64

In [47]:
final_df = pd.concat([df, ch_df], axis =1)
final_df

,zcrs0,zcrs1,zcrs2,zcrs3,#ofcrossing,C,C♯,D,D♯,E,F,F♯,G,G♯,A,A♯,B
0,0.018000,0.031625,0.035125,0.029000,1337.0,0.808428,0.652943,0.501026,0.411353,0.472592,0.516226,0.536706,0.533911,0.702158,0.749642,0.66438,0.629084
1,0.014500,0.032625,0.024375,0.025875,1148.0,0.693731,0.77702,0.510975,0.479124,0.398433,0.406758,0.462523,0.419838,0.448998,0.48601,0.679584,0.663704
2,0.016125,0.029625,0.026875,0.024750,1135.0,0.773602,0.936913,0.699597,0.42843,0.353148,0.330228,0.269443,0.336155,0.398033,0.464294,0.556676,0.604561
3,0.020625,0.035125,0.033875,0.032500,1322.0,0.878625,0.853817,0.705735,0.553189,0.554464,0.509743,0.487549,0.470062,0.471314,0.441334,0.538502,0.693372
4,0.015875,0.036875,0.038000,0.033375,1344.0,0.816637,0.730825,0.546518,0.437123,0.566929,0.457746,0.434965,0.521014,0.443097,0.467492,0.598752,0.754699
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10714,0.016250,0.050250,0.052500,0.034750,1757.0,0.774699,0.781516,0.607901,0.398823,0.347227,0.421595,0.403844,0.360063,0.382821,0.525848,0.373043,0.402533
10715,0.012625,0.034250,0.032125,0.033500,1274.0,0.801259,0.819637,0.8101,0.85162,0.660284,0.624944,0.616555,0.482968,0.585966,0.61571,0.679161,0.707453
10716,0.036875,0.035375,0.026875,0.029500,1542.0,0.475532,0.386038,0.403829,0.480387,0.602,0.372674,0.334681,0.256551,0.309038,0.494605,0.688533,0.817985
10717,0.032375,0.088250,0.097250,0.044625,3216.0,0.368217,0.548284,0.630973,0.532328,0.293125,0.427472,0.301166,0.237313,0.344682,0.398755,0.3433,0.246092


In [48]:
final_df = pd.concat([final_df, df_t], axis=1)
final_df

,zcrs0,zcrs1,zcrs2,zcrs3,#ofcrossing,C,C♯,D,D♯,E,...,tempogram_182,tempogram_183,tempogram_184,tempogram_185,tempogram_186,tempogram_187,tempogram_188,tempogram_189,tempogram_190,tempogram_191
0,0.018000,0.031625,0.035125,0.029000,1337.0,0.808428,0.652943,0.501026,0.411353,0.472592,...,-0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0
1,0.014500,0.032625,0.024375,0.025875,1148.0,0.693731,0.77702,0.510975,0.479124,0.398433,...,-0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.016125,0.029625,0.026875,0.024750,1135.0,0.773602,0.936913,0.699597,0.42843,0.353148,...,0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0
3,0.020625,0.035125,0.033875,0.032500,1322.0,0.878625,0.853817,0.705735,0.553189,0.554464,...,0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0
4,0.015875,0.036875,0.038000,0.033375,1344.0,0.816637,0.730825,0.546518,0.437123,0.566929,...,-0.0,-0.0,0.0,0.0,0.0,-0.0,-0.0,-0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10714,0.016250,0.050250,0.052500,0.034750,1757.0,0.774699,0.781516,0.607901,0.398823,0.347227,...,-0.0,0.0,0.0,-0.0,0.0,-0.0,-0.0,0.0,-0.0,0.0
10715,0.012625,0.034250,0.032125,0.033500,1274.0,0.801259,0.819637,0.8101,0.85162,0.660284,...,-0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0
10716,0.036875,0.035375,0.026875,0.029500,1542.0,0.475532,0.386038,0.403829,0.480387,0.602,...,-0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0
10717,0.032375,0.088250,0.097250,0.044625,3216.0,0.368217,0.548284,0.630973,0.532328,0.293125,...,0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0


In [49]:
final_df.to_csv(path + '/final_sp_MV.csv')